# MWS Secondary Target Selection

## MWS_SPECIAL_DDOGIANTS

Maintainers:
  - David Nidever (selection/catalog)
  - Ting Li
  - Andrew Cooper (secondary target code)

In [23]:
ls ../raw

allgiants_xm_gaia_4DESI_v0.fits


{ DESCRIPTION NEEDED }

The selection criteria are:

???

In [24]:
pwd

'/Users/apcooper/astro/data/desi/secondary/MWS_SPECIAL_DDOGIANTS_SV/1/code'

In [25]:
import os
import numpy as np
from astropy.table import Table, Column
import astropy.io.fits as fits

In [26]:
target_class   = 'MWS_SPECIAL_DDOGIANTS_SV'
input_filename = 'allgiants_xm_gaia_4DESI_v0.fits'

REF_CAT   = 'G'
REF_EPOCH = '2015.5'

In [27]:
# Relative paths
input_path  = os.path.join('..','raw',input_filename)

# Separate bright and faint output
output_path = os.path.join('..','indata','{}.fits'.format(target_class))

output_dir  = os.path.dirname(output_path)
if not os.path.exists(output_dir):
    print('Making output directory: {}'.format(os.path.abspath(output_dir)))
    os.makedirs(dir)

In [28]:
print('Reading input: {}'.format(os.path.abspath(input_path)))
d = Table.read(input_path)
print('Read {:d} rows'.format(len(d)))

Reading input: /Users/apcooper/astro/data/desi/secondary/MWS_SPECIAL_DDOGIANTS_SV/1/raw/allgiants_xm_gaia_4DESI_v0.fits
Read 291411 rows


In [29]:
print('First 3 rows:')
print(d[0:3])
print(d.colnames)

First 3 rows:
       ra              dec        ... phot_bp_mean_mag phot_rp_mean_mag
---------------- ---------------- ... ---------------- ----------------
328.519773312368 2.40651951768389 ...        16.265697        15.058078
328.530788027781 2.53707028993252 ...        19.563778        18.097618
328.592711695555 2.42055391398862 ...        15.198254        14.016314
['ra', 'dec', 'source_id', 'pmra', 'pmdec', 'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag']


Mag range: 10.67 < G < 20.88
Targets brighter than G = 16: 75258/291411


In [52]:
BRIGHT_TIME_BRIGHT_LIMIT = 16.0

print('Mag range: {:5.2f} < G < {:5.2f}'.format(d['phot_g_mean_mag'].min() ,d['phot_g_mean_mag'].max()))
print('Targets brighter than G = 16: {}/{} ({:3.2%})'.format((d['phot_g_mean_mag'] < BRIGHT_TIME_BRIGHT_LIMIT).sum(),
                                                   len(d),
                                                   (d['phot_g_mean_mag'] < BRIGHT_TIME_BRIGHT_LIMIT).sum()/len(d)
                                                  ))

bright_time_targets = d['phot_g_mean_mag'] > BRIGHT_TIME_BRIGHT_LIMIT

Mag range: 10.67 < G < 20.88
Targets brighter than G = 16: 75258/291411 (25.83%)


Adjust to secondary target data model:

In [53]:
output = d['ra','dec','source_id'][bright_time_targets]
output.rename_columns(['ra','dec','source_id'],['RA','DEC','REF_ID'])
output.add_column(Column(np.repeat(False,len(output)),'OVERRIDE'))
output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMRA'))
output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMDEC'))
output.add_column(Column(np.repeat(REF_CAT,len(output)),'REF_CAT'))
output.add_column(Column(np.repeat(REF_EPOCH,len(output)),'REF_EPOCH'))

Add header:

In [54]:
output.meta['VRAWDATA'] = int(os.path.abspath(input_path).split(os.path.sep)[-3])

In [55]:
output[0:3]

RA,DEC,REF_ID,OVERRIDE,PMRA,PMDEC,REF_CAT,REF_EPOCH
float64,float64,int64,bool,float32,float32,str1,str6
328.530788027781,2.53707028993252,2693964742599255040,False,0.0,0.0,G,2015.5
328.59327920521,2.44939161551723,2693916226648580224,False,0.0,0.0,G,2015.5
328.591951469507,2.36698433447594,2693912374062424192,False,0.0,0.0,G,2015.5


Write output:

In [56]:
print('Writing {:d} rows of output to {}'.format(len(output),os.path.abspath(output_path)))
output.write(output_path,overwrite=True)

Writing 216153 rows of output to /Users/apcooper/astro/data/desi/secondary/MWS_SPECIAL_DDOGIANTS_SV/1/indata/MWS_SPECIAL_DDOGIANTS_SV.fits
